# SETUP & AUTHENTIFIZIERUNG

In [35]:
# Bibliotheken
import requests
import base64
from pprint import pprint
import pandas as pd
import time
from tqdm import tqdm
import sys
import os

# Fügt das Hauptverzeichnis (eins höher als /notebooks) zum Suchpfad hinzu
sys.path.append(os.path.abspath(os.path.join('..')))

# # Module importieren
from src.spotify_utils import refresh_access_token, get_spotify_ids, get_artists_batch, get_tracks_batch

In [36]:
# Spotify API Credentials - passende Daten einfügen.
CLIENT_ID = your_id_here
CLIENT_SECRET = your_secret_here
refresh_token = your_token_here

# Token generieren
ACCESS_TOKEN = refresh_access_token(refresh_token, CLIENT_ID, CLIENT_SECRET)

if ACCESS_TOKEN:
    print("Access Token erfolgreich generiert.")

Access Token erfolgreich generiert.


In [37]:
# Vorbereitete Liste laden
unique_tracks = pd.read_csv("../data/interim/unique_tracks_to_enrich.csv")

# Mapping durchführen
tqdm.pandas() # Aktiviert Fortschrittsbalken für Pandas
print("Starte Suche nach Spotify IDs...")

# Suche auf jede Zeile anwenden (Funktion aufrufen)
ids = unique_tracks.apply(
    lambda row: get_spotify_ids(row['track_name'], row['artist_names'], ACCESS_TOKEN), 
    axis=1
)

# Ergebnisse in neue Spalten schreiben
unique_tracks[['track_id', 'artist_id']] = pd.DataFrame(ids.tolist(), index=unique_tracks.index)

# Zwischenspeichern 
unique_tracks.to_csv("../data/interim/unique_tracks_with_ids.csv", index=False)
print(f"Mapping fertig! {unique_tracks['track_id'].notna().sum()} IDs gefunden.")

Starte Suche nach Spotify IDs...
Mapping fertig! 1516 IDs gefunden.


In [38]:
# Vorbereitung
df_ids = pd.read_csv("../data/interim/unique_tracks_with_ids.csv").dropna(subset=['track_id', 'artist_id'])
track_ids = df_ids['track_id'].tolist()
artist_ids = df_ids['artist_id'].tolist()

all_enriched_data = []
batch_size = 50 

print(f"Starte Enrichment für {len(df_ids)} Tracks (Genres, Popularity, Release Dates)...")

# Batch-Schleife
for i in tqdm(range(0, len(track_ids), batch_size)):
    t_batch = track_ids[i:i + batch_size]
    # Artist IDs für diesen Batch (wir nehmen ein Set gegen Duplikate im Batch)
    a_batch = list(set(artist_ids[i:i + batch_size]))
    
    # API Abfragen
    tracks_res = get_tracks_batch(t_batch, ACCESS_TOKEN)
    artists_res = get_artists_batch(a_batch, ACCESS_TOKEN)
    
    # Mapping der Artist-Infos (für schnellen Zugriff)
    artist_map = {a['id']: a for a in artists_res if a}
    
    # Daten zusammenführen
    for track in tracks_res:
        # Falls ein Track im Batch komplett leer/None ist, überspringen
        if not track or 'id' not in track:
            continue
            
        main_artist_id = track['artists'][0]['id'] if track.get('artists') else None
        artist_info = artist_map.get(main_artist_id, {})

        album_info = track.get('album', {})
        release_date = album_info.get('release_date', track.get('release_date', "1900-01-01"))

        all_enriched_data.append({
                'track_id': track.get('id'),
                'track_name': track.get('name'),
                'artist_id': main_artist_id,
                'release_date': release_date, # Jetzt viel sicherer!
                'explicit': track.get('explicit', False),
                'track_popularity': track.get('popularity', 0),
                'artist_genres': "|".join(artist_info.get('genres', [])),
                'artist_followers': artist_info.get('followers', {}).get('total', 0),
                'artist_popularity': artist_info.get('popularity', 0)
        })
        
    time.sleep(1)

# Speichern
df_final = pd.DataFrame(all_enriched_data)
df_final.to_csv("../data/interim/enriched_data.csv", index=False)
print(f"Fertig! {len(df_final)} Tracks mit Artist-Metadaten angereichert.")

Starte Enrichment für 1516 Tracks (Genres, Popularity, Release Dates)...


100%|██████████| 31/31 [00:52<00:00,  1.71s/it]

Fertig! 1516 Tracks mit Artist-Metadaten angereichert.
